In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-3.5.4/spark-3.5.4-bin-hadoop3.tgz
!tar xf spark-3.5.4-bin-hadoop3.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.4-bin-hadoop3"
import findspark
findspark.init()

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

In [5]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
homesales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header = True, ignoreLeadingWhiteSpace = True)
homesales_df.createOrReplaceTempView("home_sales")
homesales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
    SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS avg_price_4bedroom
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY YEAR(date)
    ORDER BY year
""").show()

+----+------------------+
|year|avg_price_4bedroom|
+----+------------------+
|2019|          300263.7|
|2020|         298353.78|
|2021|         301819.44|
|2022|         296363.88|
+----+------------------+



In [7]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
    SELECT date_built, ROUND(AVG(price), 2) AS avg_price_3bedroom
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built
""").show()

+----------+------------------+
|date_built|avg_price_3bedroom|
+----------+------------------+
|      2010|         292859.62|
|      2011|         291117.47|
|      2012|         293683.19|
|      2013|         295962.27|
|      2014|         290852.27|
|      2015|          288770.3|
|      2016|         290555.07|
|      2017|         292676.79|
+----------+------------------+



In [9]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
    SELECT date_built, ROUND(AVG(price), 2) AS avg_price_3by3
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
""").show()

+----------+--------------+
|date_built|avg_price_3by3|
+----------+--------------+
|      2010|     285010.22|
|      2011|     276553.81|
|      2012|     307539.97|
|      2013|     303676.79|
|      2014|     298264.72|
|      2015|     297609.97|
|      2016|      293965.1|
|      2017|     280317.58|
+----------+--------------+



In [11]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price_view
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view
""")
query.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------+
|view|avg_price_view|
+----+--------------+
| 100|     1026669.5|
|  51|     788128.21|
|  52|     733780.26|
|  53|      755214.8|
|  54|     798684.82|
|  55|     771153.32|
|  56|      718176.4|
|  57|      734340.5|
|  58|     759764.65|
|  59|      791453.0|
|  60|     754939.65|
|  61|     746877.59|
|  62|     759150.14|
|  63|     711614.55|
|  64|     767036.67|
|  65|     736679.93|
|  66|      712475.0|
|  67|     737970.96|
|  68|     716785.44|
|  69|     750537.94|
+----+--------------+
only showing top 20 rows

--- 0.5813488960266113 seconds ---


In [12]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [13]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [14]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

cached_query = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price_view
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view
""")
cached_query.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------+
|view|avg_price_view|
+----+--------------+
| 100|     1026669.5|
|  51|     788128.21|
|  52|     733780.26|
|  53|      755214.8|
|  54|     798684.82|
|  55|     771153.32|
|  56|      718176.4|
|  57|      734340.5|
|  58|     759764.65|
|  59|      791453.0|
|  60|     754939.65|
|  61|     746877.59|
|  62|     759150.14|
|  63|     711614.55|
|  64|     767036.67|
|  65|     736679.93|
|  66|      712475.0|
|  67|     737970.96|
|  68|     716785.44|
|  69|     750537.94|
+----+--------------+
only showing top 20 rows

--- 0.6314523220062256 seconds ---


In [15]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
homesales_df.write.partitionBy("date_built").parquet("home_sales_partitioned")

In [16]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet("home_sales_partitioned")

In [17]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('parquet_temp_home')

In [20]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

parquet_query = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price_view
    FROM parquet_temp_home
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view
""")
parquet_query.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------+
|view|avg_price_view|
+----+--------------+
| 100|     1026669.5|
|  51|     788128.21|
|  52|     733780.26|
|  53|      755214.8|
|  54|     798684.82|
|  55|     771153.32|
|  56|      718176.4|
|  57|      734340.5|
|  58|     759764.65|
|  59|      791453.0|
|  60|     754939.65|
|  61|     746877.59|
|  62|     759150.14|
|  63|     711614.55|
|  64|     767036.67|
|  65|     736679.93|
|  66|      712475.0|
|  67|     737970.96|
|  68|     716785.44|
|  69|     750537.94|
+----+--------------+
only showing top 20 rows

--- 0.827547550201416 seconds ---


In [24]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [25]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False